<a href="https://colab.research.google.com/github/huybui28/vnstock/blob/main/api_x_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Google Sheets (https://docs.google.com/spreadsheets/d/1RlfwKXiOe7wz5lnjGmGgvjpwgSpB98TwfaGngb_cpGc/edit?gid=0#gid=0)

## Tải file khóa JSON API

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving my-project-gsheet-438409-82649f5dcedb.json to my-project-gsheet-438409-82649f5dcedb (3).json


# Cài đặt thư viện

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas
!pip install vnstock3 --upgrade
!pip install --upgrade gspread oauth2client


## Gọi thư viện API

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials  # Import đúng cách

## Gọi thư viện Vnstock3

In [ ]:
from vnstock import * #import all functions

# Cấu hình API - Sheets (URL)

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

SERVICE_ACCOUNT_FILE = '/content/my-project-gsheet-438409-82649f5dcedb.json' # Thay thế bằng đường dẫn đến tệp JSON

# Xác định phạm vi quyền truy cập
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive"
         ]

# Kết nối với API
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/my-project-gsheet-438409-82649f5dcedb.json', scope)
client = gspread.authorize(credentials)

In [ ]:
# Mở Google Sheets bằng URL
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1RlfwKXiOe7wz5lnjGmGgvjpwgSpB98TwfaGngb_cpGc/edit?gid=0#gid=0'  # Thay thế bằng URL của Google Sheets
spreadsheet = client.open_by_url(spreadsheet_url)

# Sheets: Xuất Data vào tên sheets

In [ ]:
# Chọn worksheet cụ thể
sheet = spreadsheet.worksheet("Hello")  # Thay thế bằng tên sheet nếu khác

# Lấy giá cổ phiếu (def)

1.  Hàm
2.  Kết nối API
3.  Lấy giá cổ phiếu (đóng gói)
4.  Chuyển đổi định dạng
5.  Sửa lỗi
6. Vận hành, thông tin Cổ Phiếu


In [ ]:
def update_stock_data(symbol, sheet_name="Hello"):
    """Cập nhật dữ liệu giá cổ phiếu vào Google Sheets.

    Args:
        symbol: Mã cổ phiếu (ví dụ: "VHM").
        sheet_name: Tên sheet trong Google Sheets (mặc định là "Hello").
    """

    try:
        # Kết nối với API
        credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/my-project-gsheet-438409-82649f5dcedb.json', scope)
        client = gspread.authorize(credentials)
        # Mở Google Sheets bằng URL
        spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1RlfwKXiOe7wz5lnjGmGgvjpwgSpB98TwfaGngb_cpGc/edit?gid=0#gid=0'  # Thay thế bằng URL của Google Sheets
        spreadsheet = client.open_by_url(spreadsheet_url)

        # Lấy dữ liệu giá cổ phiếu
        data = stock_intraday_data(symbol, # type: ignore
                            page_size=10000, investor_segment=True)

        # Chuyển đổi dữ liệu thành list of lists
        data_to_update = [data.columns.values.tolist()] + data.values.tolist()

        # Cập nhật dữ liệu vào sheet
        sheet.clear() # Xóa dữ liệu cũ trước khi cập nhật (có thể bỏ dòng này nếu muốn giữ dữ liệu cũ)
        sheet.update('A1', data_to_update)

        print(f"Dữ liệu giá cổ phiếu {symbol} đã được cập nhật vào sheet '{sheet_name}'!")

    except FileNotFoundError:
        print(f"Không tìm thấy tệp JSON tại: {'/content/my-project-gsheet-438409-82649f5dcedb.json'}")
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Không tìm thấy Spreadsheet: {'/content/my-project-gsheet-438409-82649f5dcedb.json'}")
    except gspread.exceptions.WorksheetNotFound:
        print(f"Không tìm thấy Worksheet: {'Hello'}")
    except Exception as e:
        print(f"Đã xảy ra lỗi với mã cổ phiếu {symbol}: {e}")

In [ ]:
if __name__ == "__main__":
    # Sử dụng hàm update_stock_data để cập nhật dữ liệu cho các mã cổ phiếu khác nhau
    update_stock_data("FPT") # Cập nhật dữ liệu FPT vào sheet "" (mặc định)

The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)


Dữ liệu giá cổ phiếu FPT đã được cập nhật vào sheet 'Hello'!


# Input Cổ Phiếu (Cần lấy)

In [ ]:
update_stock_data("FPT", sheet_name="Hello") # Cập nhật dữ liệu cổ phiếu

The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)


Dữ liệu giá cổ phiếu FPT đã được cập nhật vào sheet 'Hello'!


# Tự động cập nhật dữ liệu